# Instalación de dependencias para librerías y bases de datos

In [1]:
## only with GPU
!apt-get install --allow-change-held-packages libcudnn8=8.1.1.33-1+cuda11.2 -y
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases
!pip install scikeras[tensorflow]




Package libcudnn8 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Version '8.1.1.33-1+cuda11.2' for 'libcudnn8' was not found
  Cloning https://github.com/UN-GCPDS/python-gcpds.databases to /tmp/pip-req-build-ikkrz9kd
  Running command git clone --filter=blob:none --quiet https://github.com/UN-GCPDS/python-gcpds.databases /tmp/pip-req-build-ikkrz9kd
  Resolved https://github.com/UN-GCPDS/python-gcpds.databases to commit d174df9958b6638156dcfe03996a6307e631a6a2
  Preparing metadata (setup.py) ... done
  Created wheel for gcpds-databases: filename=gcpds_databases-0.2-py3-none-any.whl size=32972807 sha256=cad8244dbb2eba49e655fff9ed9baa4d3ffa66a3e956bbc818e9869f7abcc86b
  Stored in directory: /tmp/pip-ephem-wheel-cache-96u6gp78/wheels/ae/48/8d/edf617d5fe8f03b17aa26306a04abdfcc605b218d8e6deac83
Successfully built gcpds-databases
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.

--2025-04-28 14:54:20--  https://docs.google.com/uc?export=download&confirm=&id=1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7
Resolving docs.google.com (docs.google.com)... 173.194.203.102, 173.194.203.138, 173.194.203.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.203.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7&export=download [following]
--2025-04-28 14:54:20--  https://drive.usercontent.google.com/download?id=1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.20.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86900 (85K) [application/octet-stream]
Saving to: ‘MI_EEG_ClassMeth.zip’

MI_EEG_ClassMeth.zi 100%[===================>]  84.86K 

# Importación de librerías

In [3]:
# gcpds datasets
from gcpds.databases import GIGA_MI_ME
from gcpds.databases.BCI_Competition_IV import Dataset_2a
import gcpds.databases as loaddb

# freq filter 
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr

# general
import numpy as np
from scipy.signal import resample
import pickle
import warnings
import mne
from time import time
warnings.filterwarnings('ignore')

In [4]:
!git clone https://github.com/UN-GCPDS/eeg_models.git

import sys
sys.path.append('/kaggle/working/eeg_models')

from utils import train
import tensorflow as tf

Cloning into 'eeg_models'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (143/143), done.
Receiving objects: 100% (198/198), 117.00 KiB | 4.88 MiB/s, done.
remote: Total 198 (delta 129), reused 105 (delta 53), pack-reused 0 (from 0)
Resolving deltas: 100% (129/129), done.


2025-04-28 14:54:30.750761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745852070.954525      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745852071.009661      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Creación del modelo

In [5]:
from models.IRKCSFCnet import GFC_reg_renyi

# Parámetros específicos del modelo 
model_params = {"nb_classes":2, 
        "filters":2,
        "Chans" :64, 
        "Samples" :320,
        "dropoutRate" :0.5,
        "l1" : 0, 
        "l2" : 0,
        "alpha" :1.0,
        "filters" :2, 
        "maxnorm" :2.0,
        "maxnorm_last_layer" :0.5,
        "kernel_time_1" :25,
        "strid_filter_time_1" : 1,
        "bias_spatial" : False,
}

# Argumentos generales del modelo
model_args = {
    'model_name': 'IRKCS-FCnet',
    'nb_classes': 2,
    'autoencoder': False
}

model= GFC_reg_renyi(**model_params)
model.summary()

I0000 00:00:1745852084.880074      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745852084.880725      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)                   │ (None, 64, 320, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 64, 296, 2)          │              50 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64, 296, 2)          │               8 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 64, 296, 2)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gfc (GFC)                            │ (None, 2, 64, 64)           │               1 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ get_triu (get_triu)                  │ (None, 2, 2016, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 1, 2016, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 1, 2016, 1)          │               4 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 1, 2016, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2016)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2016)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ logits (Dense)                       │ (None, 2)                   │           4,034 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Activation)                  │ (None, 2)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,097 (16.00 KB)

 Trainable params: 4,091 (15.98 KB)

 Non-trainable params: 6 (24.00 B)

# Pruebas con GIGA

In [6]:
# Nombre de la base de datos y carga
db_name = 'GIGA_MI_ME'
db_path = '/kaggle/input/giga-science-gcpds/GIGA_MI_ME'
db = GIGA_MI_ME(db_path)

# Frecuencia de muestreo
fs = db.metadata['sampling_rate']

# Nombres de los canales EEG
eeg_ch_names = [
    'Fp1', 'Fpz', 'Fp2',
    'AF7', 'AF3', 'AFz', 'AF4', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8',
    'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
    'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8',
    'P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10',
    'PO7', 'PO3', 'POz', 'PO4', 'PO8',
    'O1', 'Oz', 'O2',
    'Iz'
]

# Definición de los bancos de frecuencia y ventana temporal
f_bank = np.array([[4., 40.]])        # Banda de frecuencia de interés
vwt = np.array([[2.5, 5]])            # Ventana temporal para la representación TF

# Representación tiempo-frecuencia
tf_repr = TimeFrequencyRpr(sfreq=fs, f_bank=f_bank, vwt=vwt)

# Argumentos para la función de carga de datos
load_args = {
    'db': db,
    'eeg_ch_names': eeg_ch_names,
    'fs': fs,
    'f_bank': f_bank,
    'vwt': vwt,
    'new_fs': 128.0,
    'tf_repr': tf_repr
}

# Selección de sujetos (excluyendo sujetos 29 y 34)
subjects = np.arange(1, db.metadata['subjects'] + 1)
subjects = np.delete(subjects, [28, 33])

In [7]:
# === Imports ===
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from sklearn.model_selection import StratifiedShuffleSplit

# === Parámetros generales ===
seed = 23
verbose = 0

# === Callbacks de entrenamiento ===
reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=30,
    verbose=verbose,
    mode='min',
    min_delta=0.01,
    min_lr=0
)

terminate_on_nan = TerminateOnNaN()

callbacks = [reduce_lr_on_plateau, terminate_on_nan]

# === Configuración de validación cruzada ===
cv_args = {
    'cv': StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=seed)
}

# === Argumentos de compilación del modelo ===
compile_args = {
    'loss': SparseCategoricalCrossentropy(),  # Alternativa: 'mse' o MeanSquaredError()
    'init_lr': 0.001
}

# === Argumentos del ajuste (fit) del modelo ===
fit_args = {
    'epochs': 500,
    'verbose': verbose,
    'callbacks': callbacks
}

In [8]:
all_results = []

for sbj in subjects[:]: 
    print('sbj = ', sbj)
    load_args['sbj'] = sbj
    results = train(model, db_name, tf_repr, load_args, cv_args, model_args, compile_args, fit_args, seed)
    all_results.append((sbj, results))

sbj =  1
Resampling from 512.000000 to 128.000000 Hz.


I0000 00:00:1745852095.217371     129 service.cc:148] XLA service 0x793fe000c400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745852095.218076     129 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745852095.218096     129 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745852095.644354     129 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745852099.344358     129 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 982ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 757ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 760ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 764ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 769ms/step
sbj =  2
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 768ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 764ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 759ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 812ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 807ms/step
sbj =  3
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 798ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 755ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 767ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 759ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 762ms/step
sbj =  4
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 766ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 757ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 757ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 800ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 775ms/step
sbj =  5
Resampling from 512.000000 to 128.000000 Hz.
2/2 

In [9]:
import pickle

with open(f'results_{model_args["model_name"]}.pkl', 'wb') as f:
    pickle.dump(all_results, f)